# Benchmark of Matrix Multiplications
On this benchmark we compare several operations using numpy, numexpr and numba (CPU&GPU).

In [20]:
import sys
import os
import numpy as np
import numexpr as ne
from numba import vectorize
import math
from functools import reduce
import pandas as pd
import bokeh
from utils import (get_number_processors, get_ram_memory, get_total_gpu_memory, 
                   get_gpu_name, get_cuda_version, get_cudnn_version, AttributeDict)

print("System version: {}".format(sys.version))
print("Numpy version: {}".format(np.__version__))
print("Pandas version: {}".format(pd.__version__))
print("Numexpr version: {}".format(ne.__version__))


%load_ext autoreload
%autoreload 2

System version: 3.6.7 |Anaconda, Inc.| (default, Oct 23 2018, 19:16:44) 
[GCC 7.3.0]
Numpy version: 1.15.4
Pandas version: 0.23.4
Numexpr version: 2.6.9
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Helper function with numpy

In [29]:
def multiply(a,b):
    return a*b

def exponential(a, b):
    return a*np.exp(b)

def sine(a, b):
    return a*np.sin(b)

# A general function that multiplies an arbitrary number of matrices
# is 28% slower than directly multiplying the factors.
# The function multiply_list is not used, just leaving it here for reference
def multiply_list(l):
    return reduce(lambda x, y: x*y, l) 

def multiply3(a, b, c):
    return a*b*c

def multiply5(a, b, c, d, e):
    return a*b*c*d*e

def exponential_sine(a, b, c):
    return a*np.exp(b)*np.sin(c)

## Helper function with numpexp

In [30]:
def ne_multiply(a,b):
    return ne.evaluate("a*b")

def ne_exponential(a, b):
    return ne.evaluate("a*exp(b)")

def ne_sine(a, b):
    return ne.evaluate("a*sin(b)")

def ne_multiply3(a, b, c):
    return ne.evaluate("a*b*c")

def ne_multiply5(a, b, c, d, e):
    return ne.evaluate("a*b*c*d*e")

def ne_exponential_sine(a, b, c):
    return ne.evaluate("a*exp(b)*sin(c)")


## Helper functions for numba
NOTE: For numba solutions, having a solution empty vector speeds up around 10%
```
r0 = np.empty((S1, S2), dtype=np.int16)
r0 = multicpu(a, b)
```
source: https://devblogs.nvidia.com/numba-python-cuda-acceleration/

In [31]:
@vectorize(["int16(int16, int16)"], target="cpu")
def multicpu(a, b):
    return a * b

@vectorize(["int16(int16, int16)"], target="cuda")
def multicuda(a, b):
    return a * b

@vectorize(["float32(float32, float32)"], target="cpu")
def multfcpu(a, b):
    return a * b

@vectorize(["float32(float32, float32)"], target="cuda")
def multfcuda(a, b):
    return a * b

@vectorize(["float32(float32, float32)"], target="cpu")
def expcpu(a, b):
    return a*math.exp(b)

@vectorize(["float32(float32, float32)"], target="cuda")
def expcuda(a, b):
    return a*math.exp(b)

@vectorize(["float32(float32, float32)"], target="cpu")
def sincpu(a, b):
    return a*math.sin(b)

@vectorize(["float32(float32, float32)"], target="cuda")
def sincuda(a, b):
    return a*math.sin(b)

@vectorize(["float32(float32, float32, float32)"], target="cpu")
def multfcpu3(a, b, c):
    return a * b * c

@vectorize(["float32(float32, float32, float32)"], target="cuda")
def multfcuda3(a, b, c):
    return a * b * c

@vectorize(["float32(float32, float32, float32, float32, float32)"], target="cpu")
def multfcpu5(a, b, c, d, e):
    return a * b * c * d * e

@vectorize(["float32(float32, float32, float32, float32, float32)"], target="cuda")
def multfcuda5(a, b, c, d, e):
    return a * b * c * d * e

@vectorize(["float32(float32, float32, float32)"], target="cpu")
def expsincpu(a, b, c):
    return a*math.exp(b)*math.sin(c)

@vectorize(["float32(float32, float32, float32)"], target="cuda")
def expsincuda(a, b, c):
    return a*math.exp(b)*math.sin(c)

## Data

In [39]:
def factors_int(s1=100, s2=100):
    a = np.random.randint(1, 5, (s1, s2), dtype=np.int16)
    b = np.random.randint(1, 10, (s1, s2), dtype=np.int16)
    return a, b

def factors_float(s1=100, s2=100):
    a = np.random.randn(s1, s2).astype(np.float32)
    b = np.random.randn(s1, s2).astype(np.float32)
    return a, b

def factors_float3(s1=100, s2=100):
    a = np.random.randn(s1, s2).astype(np.float32)
    b = np.random.randn(s1, s2).astype(np.float32)
    c = np.random.uniform(low=0, high=10, size=(s1,s2)).astype(np.float32)
    return a, b, c

def factors_float5(s1=100, s2=100):
    a = np.random.randn(s1, s2).astype(np.float32)
    b = np.random.randn(s1, s2).astype(np.float32)
    c = np.random.uniform(low=0, high=10, size=(s1,s2)).astype(np.float32)
    d = np.random.uniform(low=5, high=15, size=(s1,s2)).astype(np.float32)
    e = np.random.uniform(low=0, high=30, size=(s1,s2)).astype(np.float32)
    return a, b, c, d, e

## Benchmark

In [6]:
size_combinations=[
    (100, 100),
    (1000, 1000),
    (10000, 10000),
    (100000, 10000),
    (100000, 100000)
]

In [7]:
columns = ["n_processors",
           "cpu_memory",
           "gpu_name",
           "gpu_memory",
           "data_type",
           "size1",
           "size2",
           "operation",
           "numpy",
           "numexpr",
           "numba_cpu",
           "numba_gpu"]

In [11]:
n_processors = get_number_processors()
cpu_memory = get_ram_memory(units="Gb")
gpu_name = get_gpu_name()[0]
gpu_memory = get_total_gpu_memory(units="Gb")[0]
header = [n_processors, cpu_memory, gpu_name, gpu_memory]

In [47]:
filebase = gpu_name.replace(" ", "-")
filebase

'Tesla-V100-PCIE-16GB'

In [52]:
folder = "data"
os.makedirs(folder, exist_ok=True)

#### Integer matrix multiplication

In [24]:
df = pd.DataFrame(columns=columns)
for s1, s2 in size_combinations:
    a, b = factors_int(s1, s2)
    operation = "a*b"
    r1 = %timeit -o multiply(a,b)
    r2 = %timeit -o ne_multiply(a,b)
    r3 = %timeit -o multicpu(a,b)
    try:
        r4 = %timeit -o multicuda(a,b)
    except: # in case of Out Of Memory (OOM)
        r4 = AttributeDict()
        r4["average"] = "OOM"
        print("OOM for size ({},{})".format(s1, s2))
    row = header + [type(a[0,0]), s1, s2, operation, r1.average, r2.average, r3.average, r4.average]
    df.loc[len(df)] = row


2.15 µs ± 19.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
460 µs ± 10.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
2.08 µs ± 25.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
1.46 ms ± 16.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
214 µs ± 3.11 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
588 µs ± 13.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
211 µs ± 703 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
4.15 ms ± 61.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
87.5 ms ± 541 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
21.3 ms ± 402 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
87.2 ms ± 541 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
134 ms ± 2.78 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
875 ms ± 1.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
180 ms ± 2.77 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
87

In [ ]:
filename = filebase + operation + "_int" + ".csv"
df.to_csv(os.path.join(folder, filename), index=False)
df

#### Float matrix multiplication

In [26]:
df = pd.DataFrame(columns=columns)
for s1, s2 in size_combinations:
    a, b = factors_float(s1, s2)
    operation = "a*b"
    r1 = %timeit -o multiply(a,b)
    r2 = %timeit -o ne_multiply(a,b)
    r3 = %timeit -o multfcpu(a,b)
    try:
        r4 = %timeit -o multfcuda(a,b)
    except: # in case of Out Of Memory (OOM)
        r4 = AttributeDict()
        r4["average"] = "OOM"
        print("OOM for size ({},{})".format(s1, s2))
    row = header + [type(a[0,0]), s1, s2, operation, r1.average, r2.average, r3.average, r4.average]
    df.loc[len(df)] = row

3.04 µs ± 18.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
455 µs ± 11.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
3.33 µs ± 5.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
1.35 ms ± 5.95 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
42.6 µs ± 1.69 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
576 µs ± 17 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
423 µs ± 2.26 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
4.81 ms ± 32.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
24.5 ms ± 1.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
31.8 ms ± 242 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
177 ms ± 413 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
262 ms ± 36.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
216 ms ± 11.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
251 ms ± 8.49 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.81

In [ ]:
filename = filebase + operation + "_float" + ".csv"
df.to_csv(os.path.join(folder, filename), index=False)
df

#### Exponential matrix multiplication

In [25]:
df = pd.DataFrame(columns=columns)
for s1, s2 in size_combinations:
    a, b = factors_float(s1, s2)
    operation = "a*exp(b)"
    r1 = %timeit -o multiply(a,b)
    r2 = %timeit -o ne_multiply(a,b)
    r3 = %timeit -o multfcpu(a,b)
    try:
        r4 = %timeit -o multfcuda(a,b)
    except: # in case of Out Of Memory (OOM)
        r4 = AttributeDict()
        r4["average"] = "OOM"
        print("OOM for size ({},{})".format(s1, s2))        
    row = header + [type(a[0,0]), s1, s2, operation, r1.average, r2.average, r3.average, r4.average]
    df.loc[len(df)] = row

3.04 µs ± 19.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
451 µs ± 16.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
3.39 µs ± 29.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
1.44 ms ± 6.81 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
41.7 µs ± 1.04 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
562 µs ± 10.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
426 µs ± 6.44 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
4.76 ms ± 70.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
23.9 ms ± 511 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
31.9 ms ± 205 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
175 ms ± 834 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
256 ms ± 37.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
231 ms ± 11.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
270 ms ± 7.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.

In [ ]:
filename = filebase + operation + ".csv"
df.to_csv(os.path.join(folder, filename), index=False)
df

#### Sine matrix multiplication

In [27]:
df = pd.DataFrame(columns=columns)
for s1, s2 in size_combinations:
    a, b = factors_float(s1, s2)
    operation = "a*sin(b)"
    r1 = %timeit -o multiply(a,b)
    r2 = %timeit -o ne_multiply(a,b)
    r3 = %timeit -o multfcpu(a,b)
    try:
        r4 = %timeit -o multfcuda(a,b)
    except: # in case of Out Of Memory (OOM)
        r4 = AttributeDict()
        r4["average"] = "OOM"
        print("OOM for size ({},{})".format(s1, s2))        
    row = header + [type(a[0,0]), s1, s2, operation, r1.average, r2.average, r3.average, r4.average]
    df.loc[len(df)] = row

3.03 µs ± 9.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
476 µs ± 24.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
3.26 µs ± 2.99 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
1.37 ms ± 7.02 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
42 µs ± 915 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
576 µs ± 13.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
421 µs ± 1.74 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
4.69 ms ± 24.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
25.3 ms ± 3.65 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
32.5 ms ± 253 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
175 ms ± 814 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
255 ms ± 38.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
226 ms ± 265 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
287 ms ± 9.16 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.74 s

In [ ]:
filename = filebase + operation + ".csv"
df.to_csv(os.path.join(folder, filename), index=False)
df

#### Multiple matrix multiplication (3 factors)

In [42]:
df = pd.DataFrame(columns=columns)
for s1, s2 in size_combinations:
    a, b, c = factors_float3(s1, s2)
    operation = "a*b*c"
    r1 = %timeit -o multiply3(a,b,c)
    r2 = %timeit -o ne_multiply3(a,b,c)
    r3 = %timeit -o multfcpu3(a,b,c)
    try:
        r4 = %timeit -o multfcuda3(a,b,c)
    except: # in case of Out Of Memory (OOM)
        r4 = AttributeDict()
        r4["average"] = "OOM"
        print("OOM for size ({},{})".format(s1, s2))
    row = header + [type(a[0,0]), s1, s2, operation, r1.average, r2.average, r3.average, r4.average]
    df.loc[len(df)] = row

6.66 µs ± 9.51 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
461 µs ± 35.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
5.36 µs ± 43.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
1.82 ms ± 4.06 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
84.5 µs ± 2.91 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
620 µs ± 17.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
529 µs ± 1.36 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
6.22 ms ± 39.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
38.4 ms ± 1.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
43.2 ms ± 516 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
211 ms ± 5.35 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
318 ms ± 39.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
367 ms ± 5.14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
409 ms ± 6.44 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.0

In [ ]:
filename = filebase + operation + ".csv"
df.to_csv(os.path.join(folder, filename), index=False)
df

#### Multiple matrix multiplication (5 factors)

In [44]:
df = pd.DataFrame(columns=columns)
for s1, s2 in size_combinations:
    a, b, c, d, e = factors_float5(s1, s2)
    operation = "a*b*c*d*e"
    r1 = %timeit -o multiply5(a,b,c,d,e)
    r2 = %timeit -o ne_multiply5(a,b,c,d,e)
    r3 = %timeit -o multfcpu5(a,b,c,d,e)
    try:
        r4 = %timeit -o multfcuda5(a,b,c,d,e)
    except: # in case of Out Of Memory (OOM)
        r4 = AttributeDict()
        r4["average"] = "OOM"
        print("OOM for size ({},{})".format(s1, s2))
    row = header + [type(a[0,0]), s1, s2, operation, r1.average, r2.average, r3.average, r4.average]
    df.loc[len(df)] = row

14.5 µs ± 36.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
495 µs ± 22.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
8.77 µs ± 58.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
2.68 ms ± 216 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
158 µs ± 1.48 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
668 µs ± 14 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
732 µs ± 4.28 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
10.3 ms ± 58.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
68.2 ms ± 2.72 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
65.9 ms ± 1.34 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
338 ms ± 2.79 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
486 ms ± 41 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
654 ms ± 29 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
581 ms ± 26.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.85 s ± 6.48

MemoryError: 

In [ ]:
filename = filebase + operation + ".csv"
df.to_csv(os.path.join(folder, filename), index=False)
df

#### Exponential sine matrix multiplication

In [45]:
df = pd.DataFrame(columns=columns)
for s1, s2 in size_combinations:
    a, b, c = factors_float3(s1, s2)
    operation = "a*exp(b)*sin(c)"
    r1 = %timeit -o exponential_sine(a,b,c)
    r2 = %timeit -o ne_exponential_sine(a,b,c)
    r3 = %timeit -o expsincpu(a,b,c)
    try:
        r4 = %timeit -o expsincuda(a,b,c)
    except: # in case of Out Of Memory (OOM)
        r4 = AttributeDict()
        r4["average"] = "OOM"
        print("OOM for size ({},{})".format(s1, s2))        
    row = header + [type(a[0,0]), s1, s2, operation, r1.average, r2.average, r3.average, r4.average]
    df.loc[len(df)] = row

28 µs ± 316 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
472 µs ± 8.91 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
249 µs ± 3.26 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.85 ms ± 16 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
590 µs ± 58.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.36 ms ± 9.41 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
25 ms ± 391 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
7.12 ms ± 89.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
57.3 ms ± 665 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
100 ms ± 379 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
2.57 s ± 18.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
347 ms ± 39.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
556 ms ± 20 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
982 ms ± 14.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
25.7 s ± 217 ms per 

MemoryError: 

In [ ]:
filename = filebase + operation + ".csv"
df.to_csv(os.path.join(folder, filename), index=False)
df